In [ ]:
import pandas as pd
from typing import Tuple, List, Dict
from tabulate import tabulate
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import string
import nltk
import pandas as pd
from nltk.corpus import stopwords
from sklearn import metrics
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
import os

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
notspam = np.arange(0, 193)
spam = np.arange(0, 18)

df_list = []


for ns in notspam :
    df_tmp1 = pd.read_csv('/content/Train/notspam/{}_notspam.txt'.format(ns))
    df_tmp1.drop(df_tmp1.columns[[0]], axis = 1, inplace = True)
    f = open('/content/Train/notspam/{}_notspam.txt'.format(ns), "r")
    df_tmp1['message'] = f.read()
    df_tmp1['Label'] = 'notspam'
    df_list.append(df_tmp1)


for s in spam :
    df_tmp2 = pd.read_csv('/content/Train/spam/{}_spam.txt'.format(s))
    df_tmp2.drop(df_tmp2.columns[[0]], axis = 1, inplace = True)
    f2 = open('/content/Train/spam/{}_spam.txt'.format(s), "r")
    df_tmp2['message'] = f2.read()
    df_tmp2['Label'] = 'spam'
    df_list.append(df_tmp2)




In [ ]:
df_list[0]

,,,,,,,,,message,Label
> date : sun,"15 dec 91 02 : 25 : 02 est > : michael < mmorse @ vm1 . yorku . ca > > subject : re : 2 . 864 queries > > wlodek zadrozny asks is "" anything interesting "" said > construction "" s > np np "" . . . second","> much related : might consider construction form > has been discussed list late reduplication ? > logical sense "" john mcnamara name "" is tautologous thus",> level,"indistinguishable """,,"here ? "" . ' john mcnamara name ' is tautologous is support those logic-based semantics is irrelevant natural language . sense is tautologous ? supplies value attribute followed attribute is value . fact value name-attribute relevant entity were ' chaim shmendrik '",' john mcnamara name ' false . tautology,. ( reduplication,Subject: re : 2 . 882 s - > np np\n\n> date : ...,notspam


In [ ]:
df = pd.concat(df_list).reset_index(drop = True)
df = df.iloc[ : , 0 : 2]
df

,message,Label
0,Subject: re : 2 . 882 s - > np np\n\n> date : ...,notspam
1,Subject: s - > np + np\n\ndiscussion s - > np ...,notspam
2,Subject: 2 . 882 s - > np np\n\n. . . 's much ...,notspam
3,"Subject: gent conference\n\n"" listserv "" inter...",notspam
4,Subject: query : causatives korean\n\nanyone p...,notspam
...,...,...
206,Subject: `\n\nhello ! our naughty little city ...,spam
207,Subject: attract women ! ! ( 30740 )\n\nattrac...,spam
208,Subject: are serious . . . help retire 2 3 yea...,spam
209,"Subject: "" complete home business kit ! ! ""\n\...",spam


In [ ]:
def text_preprocess(mess):
    nopunc = [char for char in mess if char not in string.punctuation]
    nopunc = ''.join(nopunc)
    nopunc = nopunc.lower()
    nostop=[word for word in nopunc.split() if word.lower() not in stopwords.words('english') and word.isalpha()]

    return nostop

In [ ]:
spam_messages = df[df["Label"] == "spam"]["message"]
notspam_messages = df[df["Label"] == "notspam"]["message"]

In [ ]:
import nltk
nltk.download('stopwords')

# Words in spam messages
spam_words = []
for each_message in spam_messages:
    spam_words += text_preprocess(each_message)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
df["message"] = df["message"].apply(text_preprocess)
df["message"] = df["message"].agg(lambda x: " ".join(map(str, x)))
vectorizer = CountVectorizer()
bow_transformer = vectorizer.fit(df["message"])
messages_bow = bow_transformer.transform(df["message"])

In [ ]:
# TF-IDF
from sklearn.feature_extraction.text import TfidfTransformer

tfidf_transformer = TfidfTransformer().fit(messages_bow)

# Transform entire BoW into tf-idf corpus
messages_tfidf = tfidf_transformer.transform(messages_bow)
print(messages_tfidf.shape)

(211, 10343)


In [ ]:
FactorResult = pd.factorize(df["Label"])
df["Label"] = FactorResult[0]
df.head()

,message,Label
0,subject np np date sun dec est michael mmorse ...,0
1,subject np np discussion np np reminds years a...,0
2,subject np np much restrictive np np np pro qu...,0
3,subject gent conference listserv international...,0
4,subject query causatives korean anyone point b...,0


In [ ]:
# Split the dataset to train and test sets
msg_train, msg_test, label_train, label_test = train_test_split(
    messages_tfidf, df["Label"], test_size=0.2
)

In [ ]:
# Train an xgboost classifier
from xgboost import XGBClassifier

# Instantiate our model
clf = XGBClassifier()

# Fit the model to the training data
clf.fit(msg_train, label_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

In [ ]:
# Make predictions
predict_train = clf.predict(msg_test)


In [ ]:
no_labels = np.arange(0, 78)
df_noLabel = []

for nl in no_labels :
    df_unknown = pd.read_csv('/content/test/{}_unknown.txt'.format(nl))
    df_unknown.drop(df_unknown.columns[[0]], axis = 1, inplace = True)
    f = open('/content/test/{}_unknown.txt'.format(nl), "r")
    df_unknown['message'] = f.read()
    df_noLabel.append(df_unknown)


In [ ]:
df_noLabel1 = pd.concat(df_noLabel).reset_index(drop = True)
df_noLabel1 = df_noLabel1.iloc[ : , 0 : 1]

In [ ]:
df_noLabel1["message"] = df_noLabel1["message"].apply(text_preprocess)
df_noLabel1["message"] = df_noLabel1["message"].agg(lambda x: " ".join(map(str, x)))
messages_bow_NL = bow_transformer.transform(df_noLabel1["message"])

In [ ]:
# TF-IDF
from sklearn.feature_extraction.text import TfidfTransformer

tfidf_transformer = TfidfTransformer().fit(messages_bow_NL)

# Transform entire BoW into tf-idf corpus
messages_tfidf_NL = tfidf_transformer.transform(messages_bow_NL)


In [ ]:
model_pred = clf.predict(messages_tfidf_NL)
model_pred = pd.DataFrame(model_pred)
model_pred = model_pred.replace({0: 'nospam', 1: 'spam'})
model_pred = model_pred.to_csv('ANSWER.csv')
print(model_pred)



None
